# Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from warnings import filterwarnings
filterwarnings('ignore')
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import PorterStemmer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jadha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
data_names = pd.read_csv('Restaurant names and Metadata.csv')
data_reviews = pd.read_csv('Restaurant reviews.csv')

In [3]:
print(data_names.shape)
data_names.sample(5)

(105, 6)


,Name,Links,Cost,Collections,Cuisines,Timings
27,"3B's - Buddies, Bar & Barbecue",https://www.zomato.com/hyderabad/3bs-buddies-b...,"1,100","Barbecue & Grill, Live Sports Screenings","North Indian, Mediterranean, European","12 Noon to 4 PM, 6:30 PM to 11:30 PM"
67,Yum Yum Tree - The Arabian Food Court,https://www.zomato.com/hyderabad/yum-yum-tree-...,"1,200",Food Hygiene Rated Restaurants in Hyderabad,"North Indian, Hyderabadi",12 Noon to 12 Midnight
39,Deli 9 Bistro,https://www.zomato.com/hyderabad/deli-9-bistro...,700,NaN,"Cafe, Continental, Desserts",12 Noon to 10:30 PM
101,Sweet Basket,https://www.zomato.com/hyderabad/sweet-basket-...,200,NaN,"Bakery, Mithai","10 AM to 10 PM (Mon-Thu), 8 AM to 10:30 PM (Fr..."
92,Collage - Hyatt Hyderabad Gachibowli,https://www.zomato.com/hyderabad/collage-hyatt...,"2,800",Sunday Brunches,"Continental, Italian, North Indian, Chinese, A...",24 Hours (Mon-Sun)


In [4]:
print(data_reviews.shape)
data_reviews.sample(5)

(10000, 7)


,Restaurant,Reviewer,Review,Rating,Metadata,Time,Pictures
4414,La La Land - Bar & Kitchen,Subha Mahapatra,The most horrible and disgusting place I have ...,1,"29 Reviews , 54 Followers",4/8/2019 0:32,0
5929,KS Bakers,Msrinuhyd,"Bad experience, I have ordered for cake a day ...",1,1 Review,7/11/2018 13:24,0
9712,Zega - Sheraton Hyderabad Hotel,Dheeraj Goud,Night life and Saturday weekend parties are fu...,5,"3 Reviews , 55 Followers",4/3/2019 12:45,0
2157,Jonathan's Kitchen - Holiday Inn Express & Suites,Rushikesh Reddy,Average food not tasty enough and not worth th...,2,"2 Reviews , 35 Followers",3/8/2019 22:56,1
9523,Desi Bytes,Rithi,Good food,5,2 Reviews,9/12/2018 12:35,0


In [5]:
# take overlook on missing values and data types of individual columns
data_names.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105 entries, 0 to 104
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Name         105 non-null    object
 1   Links        105 non-null    object
 2   Cost         105 non-null    object
 3   Collections  51 non-null     object
 4   Cuisines     105 non-null    object
 5   Timings      104 non-null    object
dtypes: object(6)
memory usage: 5.0+ KB


In [6]:
# Counting the unique values
data_names.nunique()

Name           105
Links          105
Cost            29
Collections     42
Cuisines        92
Timings         77
dtype: int64

In [7]:
# take overlook on missing values and data types of individual columns
data_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Restaurant  10000 non-null  object
 1   Reviewer    9962 non-null   object
 2   Review      9955 non-null   object
 3   Rating      9962 non-null   object
 4   Metadata    9962 non-null   object
 5   Time        9962 non-null   object
 6   Pictures    10000 non-null  int64 
dtypes: int64(1), object(6)
memory usage: 547.0+ KB


In [8]:
# Counting the unique values
data_reviews.nunique()

Restaurant     100
Reviewer      7446
Review        9364
Rating          10
Metadata      2477
Time          9782
Pictures        36
dtype: int64

heare in data_reviews the restaurant column will shose the name of that perticular restaurant. so that to mearge the both data frames i am renaming the column 'Restaurent' as 'Name'

In [9]:
# Renaming the restaurant name column with the same value as in the other data set:
data_reviews = data_reviews.rename(columns={'Restaurant': 'Name'})
data_reviews.head()

,Name,Reviewer,Review,Rating,Metadata,Time,Pictures
0,Beyond Flavours,Rusha Chakraborty,"The ambience was good, food was quite good . h...",5,"1 Review , 2 Followers",5/25/2019 15:54,0
1,Beyond Flavours,Anusha Tirumalaneedi,Ambience is too good for a pleasant evening. S...,5,"3 Reviews , 2 Followers",5/25/2019 14:20,0
2,Beyond Flavours,Ashok Shekhawat,A must try.. great food great ambience. Thnx f...,5,"2 Reviews , 3 Followers",5/24/2019 22:54,0
3,Beyond Flavours,Swapnil Sarkar,Soumen das and Arun was a great guy. Only beca...,5,"1 Review , 1 Follower",5/24/2019 22:11,0
4,Beyond Flavours,Dileep,Food is good.we ordered Kodi drumsticks and ba...,5,"3 Reviews , 2 Followers",5/24/2019 21:37,0


In [10]:
# Merging the two data sets:
df = pd.merge(data_reviews, data_names, how='left', on='Name')
df.head(3)

,Name,Reviewer,Review,Rating,Metadata,Time,Pictures,Links,Cost,Collections,Cuisines,Timings
0,Beyond Flavours,Rusha Chakraborty,"The ambience was good, food was quite good . h...",5,"1 Review , 2 Followers",5/25/2019 15:54,0,https://www.zomato.com/hyderabad/beyond-flavou...,800,"Food Hygiene Rated Restaurants in Hyderabad, C...","Chinese, Continental, Kebab, European, South I...","12noon to 3:30pm, 6:30pm to 11:30pm (Mon-Sun)"
1,Beyond Flavours,Anusha Tirumalaneedi,Ambience is too good for a pleasant evening. S...,5,"3 Reviews , 2 Followers",5/25/2019 14:20,0,https://www.zomato.com/hyderabad/beyond-flavou...,800,"Food Hygiene Rated Restaurants in Hyderabad, C...","Chinese, Continental, Kebab, European, South I...","12noon to 3:30pm, 6:30pm to 11:30pm (Mon-Sun)"
2,Beyond Flavours,Ashok Shekhawat,A must try.. great food great ambience. Thnx f...,5,"2 Reviews , 3 Followers",5/24/2019 22:54,0,https://www.zomato.com/hyderabad/beyond-flavou...,800,"Food Hygiene Rated Restaurants in Hyderabad, C...","Chinese, Continental, Kebab, European, South I...","12noon to 3:30pm, 6:30pm to 11:30pm (Mon-Sun)"


# Data cleaning

In [11]:
# Dropping the columns which I am not going to use:
df.drop(['Reviewer', 'Time', 'Pictures', 'Links', 'Collections'], axis=1, inplace=True)
df.head(3)

,Name,Review,Rating,Metadata,Cost,Cuisines,Timings
0,Beyond Flavours,"The ambience was good, food was quite good . h...",5,"1 Review , 2 Followers",800,"Chinese, Continental, Kebab, European, South I...","12noon to 3:30pm, 6:30pm to 11:30pm (Mon-Sun)"
1,Beyond Flavours,Ambience is too good for a pleasant evening. S...,5,"3 Reviews , 2 Followers",800,"Chinese, Continental, Kebab, European, South I...","12noon to 3:30pm, 6:30pm to 11:30pm (Mon-Sun)"
2,Beyond Flavours,A must try.. great food great ambience. Thnx f...,5,"2 Reviews , 3 Followers",800,"Chinese, Continental, Kebab, European, South I...","12noon to 3:30pm, 6:30pm to 11:30pm (Mon-Sun)"


In [12]:
# Changing cost and rating columns data types:
df['Cost'] = df['Cost'].str.replace(',', '').astype(int)
df['Rating'] = df['Rating'].str.replace('Like', '1').astype(float)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Name      10000 non-null  object 
 1   Review    9955 non-null   object 
 2   Rating    9962 non-null   float64
 3   Metadata  9962 non-null   object 
 4   Cost      10000 non-null  int32  
 5   Cuisines  10000 non-null  object 
 6   Timings   9900 non-null   object 
dtypes: float64(1), int32(1), object(5)
memory usage: 585.9+ KB


In [13]:
print('No of data inputs:', len(df))
print('no of NaN values for each column:\n')
print(df.isnull().sum())

No of data inputs: 10000
no of NaN values for each column:

Name          0
Review       45
Rating       38
Metadata     38
Cost          0
Cuisines      0
Timings     100
dtype: int64


Rating column is important for recommender system. So I am not going to drop those 38 NaN values.
I will fill those NaN values with each restaurants' mean rating value.

In [14]:
# Finding missing Rating values:
df['Name'][df['Rating'].isnull() == True].value_counts()

American Wild Wings    23
Arena Eleven           15
Name: Name, dtype: int64

In [15]:
print('Mean of Rating for American Wild Wings: ', df['Rating'][df['Name'] == 'American Wild Wings'].mean())
print('Mean of Rating for Arena Eleven: ', df['Rating'][df['Name'] == 'Arena Eleven'].mean())
print('Overall Mean of Ratings: ', df['Rating'].mean())

Mean of Rating for American Wild Wings:  3.9740259740259742
Mean of Rating for Arena Eleven:  4.117647058823529
Overall Mean of Ratings:  3.6007829753061635


here,We can oberve that mean value for missing rating value should be 4 (3.9 and 4.1 for each restaurant).Let's fill those restaurants missing rating values.

In [16]:
df['Rating'].fillna(4, inplace=True)

# Changing NaN reviews by '-' becouse it will not create any probleme and NAN values can be handeled easily
df['Review'] = df['Review'].fillna('-')
df.isnull().sum()

Name          0
Review        0
Rating        0
Metadata     38
Cost          0
Cuisines      0
Timings     100
dtype: int64

# Text Preprocessig and Cleaning

We will be using 'Review' and 'Cuisines' feature'in order to create a recommender system.
So we need to prepare and clean the text in those columns.

In [17]:
# sample examples of these columns before text processing:
df[['Review', 'Cuisines']].head(5)

,Review,Cuisines
0,"The ambience was good, food was quite good . h...","Chinese, Continental, Kebab, European, South I..."
1,Ambience is too good for a pleasant evening. S...,"Chinese, Continental, Kebab, European, South I..."
2,A must try.. great food great ambience. Thnx f...,"Chinese, Continental, Kebab, European, South I..."
3,Soumen das and Arun was a great guy. Only beca...,"Chinese, Continental, Kebab, European, South I..."
4,Food is good.we ordered Kodi drumsticks and ba...,"Chinese, Continental, Kebab, European, South I..."


In [18]:
df.sample(5)

,Name,Review,Rating,Metadata,Cost,Cuisines,Timings
7022,Urban Asia - Kitchen & Bar,Food - 4\nService - 3\n\nThe food is nice but ...,3.0,"24 Reviews , 22 Followers",1100,"Asian, Thai, Chinese, Sushi, Momos","12 Noon to 3 PM, 7 PM to 11 PM"
4757,Shanghai Chef 2,"I am frequent here, like their noodles varieti...",4.0,"6 Reviews , 4 Followers",800,"Chinese, Thai, Indonesian, Asian",12 Noon to 11 PM
1677,NorFest - The Dhaba,"Loved the service, although their other outlet...",3.0,"20 Reviews , 424 Followers",550,North Indian,12 Noon to 10:30 PM
2830,Behrouz Biryani,Biryani aroma is good. But aroma is too strong...,2.0,"9 Reviews , 8 Followers",600,"Mughlai, Biryani",10 AM to 1 AM
6039,PourHouse7,"Went to this place, service was excellent, Tit...",5.0,"10 Reviews , 19 Followers",1200,"North Indian, Continental, Chinese, Italian","12 Noon to 12 Midnight (Mon-Thu, Sun), 12 Noon..."


In [19]:
# Define symbols to be replaced by space
replace_space = re.compile('[/(){}\[\]\|@,;]')
# Define symbols to be removed
remove_symbols = re.compile('[^0-9a-z #+_]')
# Define stopwords
stopwords = set(stopwords.words('english'))


def text_preprocessing(text):
    # Lowercase all the letters
    text = text.lower()
    
    # Replace these symbols with space
    text = replace_space.sub(' ', text)
    
    # Remove these symbols
    text = remove_symbols.sub('', text)
    
    # Remove stopwords
    text = ' '.join(word for word in text.split() if word not in stopwords)
    
    return text

In [20]:
df['Review'] = df['Review'].apply(text_preprocessing)
df['Cuisines'] = df['Cuisines'].apply(text_preprocessing)

In [21]:
# Columns after processed:
df[['Review','Cuisines']].head(5)

,Review,Cuisines
0,ambience good food quite good saturday lunch c...,chinese continental kebab european south india...
1,ambience good pleasant evening service prompt ...,chinese continental kebab european south india...
2,must try great food great ambience thnx servic...,chinese continental kebab european south india...
3,soumen das arun great guy behavior sincerety g...,chinese continental kebab european south india...
4,food goodwe ordered kodi drumsticks basket mut...,chinese continental kebab european south india...


In [22]:
stemmer = PorterStemmer()

In [23]:
# stemming

def stem_word(text):
    txt = " ".join([stemmer.stem(w) for w in text.split()])
    return txt

In [24]:
df['Review'] = df['Review'].map(lambda x: stem_word(x))
df['Cuisines'] = df['Cuisines'].map(lambda x: stem_word(x))

In [25]:
df[['Review','Cuisines']].head(5)

,Review,Cuisines
0,ambienc good food quit good saturday lunch cos...,chines continent kebab european south indian n...
1,ambienc good pleasant even servic prompt food ...,chines continent kebab european south indian n...
2,must tri great food great ambienc thnx servic ...,chines continent kebab european south indian n...
3,soumen da arun great guy behavior sincereti go...,chines continent kebab european south indian n...
4,food goodw order kodi drumstick basket mutton ...,chines continent kebab european south indian n...


In [26]:
# Changing data set index by restaurant name
df.set_index('Name', inplace=True)

# Saving indexes in a series
indices = pd.Series(df.index)

# Creating tf-idf matrix
tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['Review'])

# Calculating cosine similarities
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

In [27]:
def recommend(name, cosine_similarities = cosine_similarities):
    
    # Create a list to put top 10 restaurants
    recommend_restaurant = []
    
    # Find the index of the hotel entered
    idx = indices[indices == name].index[0]
    
    # Find the restaurants with a similar cosine-sim value and order them from bigges number
    score_series = pd.Series(cosine_similarities[idx]).sort_values(ascending=False)
    
    # Extract top 30 restaurant indexes with a similar cosine-sim value
    top30_indexes = list(score_series.iloc[0:31].index)
    
    # Names of the top 30 restaurants
    for each in top30_indexes:
        recommend_restaurant.append(list(df.index)[each])
    
    # Creating the new data set to show similar restaurants
    df_new = pd.DataFrame(columns=['Cuisines', 'Rating', 'Cost', 'Timings'])
    
    # Create the top 30 similar restaurants with some of their columns
    for each in recommend_restaurant:
        df_new = df_new.append(pd.DataFrame(df[['Cuisines','Rating', 'Cost', 'Timings']][df.index == each].sample()))
    
    # Drop the same named restaurants and sort only the top 10 by the highest rating
    df_new = df_new.drop_duplicates(subset=['Cuisines','Rating', 'Cost'], keep=False)
    df_new = df_new.sort_values(by='Rating', ascending=False).head(7)
    
    print('TOP %s RESTAURANTS LIKE %s WITH SIMILAR REVIEWS: ' % (str(len(df_new)), name))
    
    return df_new

In [28]:
# HERE IS A RANDOM RESTAURANT. LET'S SEE THE DETAILS ABOUT THIS RESTAURANT:
df[df.index == 'Karachi Bakery'].head(1)

,Review,Rating,Metadata,Cost,Cuisines,Timings
Name,,,,,,
Karachi Bakery,well know famou karachi biscuit whole rang stu...,5.0,"207 Reviews , 400 Followers",500,bakeri fast food dessert,10 AM to 10 PM


In [29]:
# LET'S SEE WHAT ARE WE GOING TO BE RECOMMENDED:
recommend('Karachi Bakery')

TOP 7 RESTAURANTS LIKE Karachi Bakery WITH SIMILAR REVIEWS: 


,Cuisines,Rating,Cost,Timings
Yum Yum Tree - The Arabian Food Court,north indian hyderabadi,5.0,1200,12 Noon to 12 Midnight
Labonel,bakeri dessert,4.5,1000,"11 AM to 8 PM (Mon-Sat), Sun Closed"
Amul,ice cream dessert,4.0,150,10 AM to 5 AM
Tandoori Food Works,north indian burger kebab,4.0,500,"12noon to 12midnight (Mon, Tue, Wed, Thu, Sun)..."
Labonel,bakeri dessert,4.0,1000,"11 AM to 8 PM (Mon-Sat), Sun Closed"
Karachi Bakery,bakeri fast food dessert,3.0,500,10 AM to 10 PM
KS Bakers,bakeri dessert fast food,3.0,200,9 AM to 10 PM


# heare are the recomended 7 hottels enjoy your food.....!